## LSTM sample

In [ ]:
symbol = "AAP"

### imports

In [ ]:
from datetime import datetime, date, timedelta
import alpaca_trade_api as tradeapi
import numpy as np
import matplotlib.pyplot as plt

### Load data

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
ohlc_data = api.polygon.historic_agg_v2(
    symbol,
    1,
    "minute",
    _from=str(date.today() - timedelta(days=7)),
    to=str(date.today()),
).df

In [ ]:
x_dates = ohlc_data.index
ohlc_data['date'] = np.arange(start = 0, stop = len(ohlc_data), step = 1, dtype='int')
ohlc_data=ohlc_data.set_index(ohlc_data.date)

In [ ]:
ohlc_data.close.plot()
plt.show()

### Pre-processing

In [ ]:
ohlc_data["delta"] = ohlc_data.close.diff()
ohlc_data["delta_pct"] = ohlc_data.close.pct_change()

In [ ]:
ohlc_data.delta.plot()
plt.show()
ohlc_data.delta_pct.plot()
plt.show()
ohlc_data.delta_pct.plot()